In [2]:
using JuMP, HiGHS
#warehouses
numWarehouses = 3
#customers
numCustomers = 3

#Cost of shipping 1 unit from warehouse to customer
costs = [15 35 25;

10 50 40;

20 40 30]

#Units each warehouse can provide
supply = [30 30 30]

#Units each customer need

demand = [40 50 40]

#Cst for each unit of unmet demand
penalty = [70 75 65]

1×3 Matrix{Int64}:
 70  75  65

In [6]:

model2= Model(HiGHS. Optimizer)

#represents
#the amount of units sent from warehouse i to customer j

@variable(model2, x[1:numWarehouses,1:numCustomers] >= 0)

#Units that fell short for each customer's demand

@variable(model2, y[1:numCustomers] >= 0)

# each i can provide at most supply[i] units
@constraint(model2, supplyconstraint[i in 1:numWarehouses], sum(x[i,j] for j in 1:numCustomers) <= supply[i])

# each j requires at Least demand[{[j] units, with unmet demand to balancing the transportation
@constraint(model2, demandconstraint[j in 1:numCustomers], sum(x[i,j] for i in 1:numWarehouses) + y[j] >= demand[j])

#Minimized Cost of sending from warehouse i to customer j is given by costs[{i,j] with the penalty costs
@objective(model2, Min, sum(sum(costs[i,j]*x[i,j] for j in 1:numCustomers) for i in 1:numWarehouses) + sum(penalty[j]* y[j] for j in 1:numCustomers) )

print(model2)


In [7]:
optimize!(model2)

Presolving model
6 rows, 12 cols, 21 nonzeros
6 rows, 12 cols, 21 nonzeros
Presolve : Reductions: rows 6(-0); columns 12(-0); elements 21(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(130) 0s
          9     4.9500000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 9
Objective value     :  4.9500000000e+03
HiGHS run time      :          0.00


In [8]:
# units sent fro warehouse to customer
println(value.(x))
# units that fell short
println(value.(y))

# smallest cost
println(objective_value(model2))

[0.0 0.0 30.0; 30.0 0.0 0.0; 10.0 20.0 0.0]
[0.0, 30.0, 10.0]
4950.0
